<a href="https://colab.research.google.com/github/tmdoi/small-Japanese-LLM-compare/blob/main/benchMarkOrginal_v02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install "transformers>=4.43" accelerate torch --upgrade
!pip -q install pandas sacrebleu rouge-score fugashi ipadic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 131.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 60.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 697.9/697.9 kB 30.1 MB/s eta 0:00:00


In [2]:
# === (Colab セル1) セットアップ & モデル読込 ===
try:
    import google.colab  # noqa: F401
    IN_COLAB = True
except Exception:
    IN_COLAB = False

import sys, subprocess, math, time, re
def pip_install(pkgs):
    cmd = [sys.executable, "-m", "pip", "install", "-q", "--upgrade"] + pkgs
    print("Installing:", " ".join(pkgs))
    subprocess.check_call(cmd)

# 必要に応じて有効化してください（初回実行時など）
# pip_install(["transformers>=4.43", "accelerate", "torch", "pandas", "sacrebleu", "rouge-score", "fugashi", "ipadic"])

# ---- 以降 Python 本体 ----
import torch, pandas as pd
from dataclasses import dataclass
from typing import List, Dict, Any, Tuple
from transformers import AutoModelForCausalLM, AutoTokenizer
from rouge_score import rouge_scorer
import sacrebleu
import numpy as np
import random

# 乱数固定（再現性の一助）
random.seed(0)
np.random.seed(0)
if torch.cuda.is_available():
    torch.manual_seed(0)
    torch.cuda.manual_seed_all(0)

# 比較対象モデル（必要に応じて変更可）
MODELS = {
    "RakutenAI-2.0-mini-instruct": "Rakuten/RakutenAI-2.0-mini-instruct",
    "TinySwallow-1.5B-Instruct":   "SakanaAI/TinySwallow-1.5B-Instruct",
}

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

@dataclass
class GenConfig:
    max_new_tokens: int = 256
    temperature: float = 0.0   # 再現性重視
    top_p: float = 1.0
    do_sample: bool = False
    num_beams: int = 1

GENCFG = GenConfig()

def load_model(repo_id: str) -> Tuple[AutoTokenizer, AutoModelForCausalLM]:
    tok = AutoTokenizer.from_pretrained(repo_id, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        repo_id,
        torch_dtype="auto",
        device_map="auto",   # ColabのGPUに自動割当
    )
    return tok, model

def chat_generate(tokenizer, model, messages: List[Dict[str, str]], cfg: GenConfig = GENCFG):
    # 各モデルのchatテンプレートを利用
    input_ids = tokenizer.apply_chat_template(
        messages, add_generation_prompt=True, return_tensors="pt"
    ).to(model.device)

    attn = None
    if tokenizer.pad_token_id is not None:
        attn = input_ids.ne(tokenizer.pad_token_id).long()

    t0 = time.perf_counter()
    with torch.no_grad():
        out_ids = model.generate(
            input_ids,
            max_new_tokens=cfg.max_new_tokens,
            do_sample=cfg.do_sample,
            temperature=cfg.temperature,
            top_p=cfg.top_p,
            num_beams=cfg.num_beams,
            attention_mask=attn,
            pad_token_id=tokenizer.eos_token_id,
        )
    dt = time.perf_counter() - t0
    gen_ids = out_ids[:, input_ids.shape[1]:]
    text = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)[0].strip()
    toks = gen_ids.shape[1]
    tps = toks / dt if dt > 0 else float("nan")
    return text, {"latency_sec": dt, "gen_tokens": toks, "tok_per_sec": tps}

# タスク定義（自動採点可能なもの中心）
TASKS = [
    {
        "name": "JA-QA: 富士山の標高",
        "messages": [
            {"role":"system","content":"あなたは有能な日本語アシスタントです。"},
            {"role":"user","content":"富士山の標高は？数値と単位で簡潔に答えてください。"}
        ],
        # 3776 を数値として含めれば正解扱い（ゆるい判定）
        "judge": lambda x: ("3776" in re.sub(r"[^\d]", "", x)) or ("3,776" in x) or ("3776 m" in x) or ("3776メートル" in x),
    },
    {
        "name": "算数: 12×(7+5)",
        "messages": [
            {"role":"system","content":"あなたは計算に正確です。"},
            {"role":"user","content":"12×(7+5) の結果だけを半角数字で答えてください。"}
        ],
        "judge": lambda x: "144" in re.sub(r"[^\d\-]", "", x),
    },
    {
        "name": "要約: 5文→1文",
        "messages": [
            {"role":"system","content":"与えられた段落を1文で要約してください。"},
            {"role":"user","content":
             "奈良公園には多くのシカが生息し、観光客に人気です。"
             "近年は観光客の増加に伴い、エサの与え方やごみ問題が課題となっています。"
             "地元自治体はルール啓発と清掃活動を強化しています。"
             "一方で来園者のマナー向上には時間がかかるとの指摘もあります。"
             "持続可能な観光の実現に向け、地域と来訪者の協力が求められています。"
            }
        ],
        "ref": "奈良公園のシカと観光をめぐる課題に対し、自治体と来訪者の協力による持続可能な観光の実現が求められている。",
        "rougeL": True
    },
    {
        "name": "翻訳: EN→JA",
        "messages": [
            {"role":"system","content":"次の英文を自然な日本語に翻訳してください。"},
            {"role":"user","content":"Edge-friendly small LLMs enable private, low-latency applications without relying on cloud services."}
        ],
        "ref": "エッジ向けの小型LLMは、クラウドサービスに依存せずにプライバシーに配慮した低遅延アプリケーションを可能にする。",
        "bleu": True
    },
]

# ROUGE-L スコアラー（※ベンチマーク側でも参照）
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=False)

# モデル／トークナイザを事前ロードして保持（ベンチマーク側で再利用）
print("\n== Loading models ==")
LOADED = {}  # { model_name: (tokenizer, model) }
for name, repo in MODELS.items():
    print(f"Loading: {name} ({repo})")
    tok, mdl = load_model(repo)
    LOADED[name] = (tok, mdl)

print("Loaded:", list(LOADED.keys()))


Device: cuda

== Loading models ==
Loading: RakutenAI-2.0-mini-instruct (Rakuten/RakutenAI-2.0-mini-instruct)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/755k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Loaded: ['RakutenAI-2.0-mini-instruct', 'TinySwallow-1.5B-Instruct']


In [4]:
# === (Colab セル2) ベンチマーク実行 ===
import math, numpy as np, pandas as pd
import sacrebleu

# セル1で定義・ロード済みの以下を利用します:
# - LOADED: { model_name: (tokenizer, model) }
# - TASKS: タスクリスト
# - scorer: ROUGE-L スコアラー
# - chat_generate(): 生成関数
# - GENCFG: 生成設定

def evaluate_one_loaded(model_name: str, tok_mdl):
    tok, mdl = tok_mdl
    rows = []
    for task in TASKS:
        out, stats = chat_generate(tok, mdl, task["messages"])
        row = {
            "model": model_name,
            "task": task["name"],
            "output": out,
            **stats
        }
        if "judge" in task:
            row["pass@1"] = bool(task["judge"](out))
        if task.get("rougeL"):
            r = scorer.score(task["ref"], out)["rougeL"].fmeasure
            row["ROUGE-L"] = r
        if task.get("bleu"):
            bleu = sacrebleu.corpus_bleu([out], [[task["ref"]]]).score
            row["BLEU"] = bleu
        rows.append(row)
        print(task["name"])
    return pd.DataFrame(rows)

# 実行
all_dfs = []
for name, tok_mdl in LOADED.items():
    print(f"\n== Evaluating {name} ==")
    df = evaluate_one_loaded(name, tok_mdl)
    display(df[["model","task","pass@1","ROUGE-L","BLEU","latency_sec","tok_per_sec","output"]])
    all_dfs.append(df)

summary = pd.concat(all_dfs, ignore_index=True)

# 集計（タスク別の平均）
def safe_mean(xs):
    xs = [x for x in xs if x is not None and not (isinstance(x, float) and math.isnan(x))]
    return float(np.mean(xs)) if xs else float("nan")

report = []
for m in summary["model"].unique():
    sub = summary[summary["model"]==m]
    pass_mean = safe_mean([1.0 if x is True else (0.0 if x is False else None) for x in sub.get("pass@1", []).tolist()])
    rouge_mean = safe_mean(sub.get("ROUGE-L", []).tolist())
    bleu_mean  = safe_mean(sub.get("BLEU", []).tolist())
    tps_mean   = safe_mean(sub.get("tok_per_sec", []).tolist())
    lat_mean   = safe_mean(sub.get("latency_sec", []).tolist())
    report.append({
        "model": m,
        "pass@1(mean)": pass_mean,
        "ROUGE-L(mean)": rouge_mean,
        "BLEU(mean)": bleu_mean,
        "tok_per_sec(mean)": tps_mean,
        "latency_sec(mean)": lat_mean
    })

print("\n== Summary ==")
display(pd.DataFrame(report))

# 生成長・温度などを変更したい場合の例（任意）
# GENCFG.max_new_tokens = 128
# GENCFG.temperature = 0.7
# GENCFG.do_sample = True
# print("New GenConfig:", GENCFG)



== Evaluating RakutenAI-2.0-mini-instruct ==


,model,task,pass@1,ROUGE-L,BLEU,latency_sec,tok_per_sec,output
0,RakutenAI-2.0-mini-instruct,JA-QA: 富士山の標高,True,NaN,NaN,0.847999,17.688694,"富士山の標高は3,776メートルです。"
1,RakutenAI-2.0-mini-instruct,算数: 12×(7+5),True,NaN,NaN,0.907494,26.446462,12 × (7 + 5) = 12 × 12 = 144
2,RakutenAI-2.0-mini-instruct,要約: 5文→1文,NaN,0.0,NaN,2.625954,33.511634,奈良公園に生息するシカは観光客に人気がありますが、近年の観光客の増加に伴い、エサの与え方やご...
3,RakutenAI-2.0-mini-instruct,翻訳: EN→JA,NaN,NaN,0.0,0.979440,33.692732,エッジ対応の小さなLLMは、クラウドサービスに頼ることなく、プライベートで低遅延のアプリケー...



== Evaluating TinySwallow-1.5B-Instruct ==


,model,task,pass@1,ROUGE-L,BLEU,latency_sec,tok_per_sec,output
0,TinySwallow-1.5B-Instruct,JA-QA: 富士山の標高,True,NaN,NaN,0.430158,23.247265,"3,776メートル"
1,TinySwallow-1.5B-Instruct,算数: 12×(7+5),False,NaN,NaN,0.225755,17.718319,94
2,TinySwallow-1.5B-Instruct,要約: 5文→1文,NaN,0.0,NaN,3.860419,25.903926,奈良公園では、シカへの餌やりやゴミ問題など、観光客による影響が課題となっており、地元自治体は...
3,TinySwallow-1.5B-Instruct,翻訳: EN→JA,NaN,NaN,0.0,2.033917,21.633136,エッジフレンドリーな小型LLMは、クラウドサービスに依存せずにプライベートで低遅延のアプリケ...



== Summary ==


,model,pass@1(mean),ROUGE-L(mean),BLEU(mean),tok_per_sec(mean),latency_sec(mean)
0,RakutenAI-2.0-mini-instruct,1.0,0.0,0.0,27.834881,1.340222
1,TinySwallow-1.5B-Instruct,0.5,0.0,0.0,22.125662,1.637562
